In [1]:
!pip install ultralytics mediapipe opencv-python qrcode numpy


   ---------------------------------------- 0/2 [qrcode]
   ---------------------------------------- 0/2 [qrcode]
   ---------------------------------------- 0/2 [qrcode]
   ---------------------------------------- 0/2 [qrcode]
   ---------------------------------------- 0/2 [qrcode]
   ---------------------------------------- 0/2 [qrcode]
  Attempting uninstall: protobuf
   ---------------------------------------- 0/2 [qrcode]
    Found existing installation: protobuf 5.29.4
   ---------------------------------------- 0/2 [qrcode]
   -------------------- ------------------- 1/2 [protobuf]
    Uninstalling protobuf-5.29.4:
   -------------------- ------------------- 1/2 [protobuf]
      Successfully uninstalled protobuf-5.29.4
   -------------------- ------------------- 1/2 [protobuf]
   -------------------- ------------------- 1/2 [protobuf]
   -------------------- ------------------- 1/2 [protobuf]
   -------------------- ------------------- 1/2 [protobuf]
   -------------------- --

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.
grpcio-status 1.69.0 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 4.25.8 which is incompatible.


In [ ]:
import cv2
import numpy as np
import mediapipe as mp
from ultralytics import YOLO
import qrcode
import time
import threading
import random

# ---------- Configuration ----------
CLUB_NAME = "SoDA"
SIGNUP_URL = "https://your-club-signup.example.com"  # Replace with actual link
BADGE_DISPLAY_SECONDS = 3
CONFETTI_COUNT = 100

# ---------- Helpers ----------
def generate_qr_image(data, size=150):
    qr = qrcode.QRCode(box_size=2, border=1)
    qr.add_data(data)
    qr.make(fit=True)
    img = qr.make_image(fill_color="black", back_color="white").convert("RGB")
    img = np.array(img)
    img = cv2.resize(img, (size, size), interpolation=cv2.INTER_AREA)
    return img

def draw_confetti(frame, confetti_particles):
    h, w = frame.shape[:2]
    new_particles = []
    for (x, y, vy, color, life) in confetti_particles:
        cv2.circle(frame, (int(x), int(y)), 5, color, -1)
        y += vy
        life -= 1
        if y < h and life > 0:
            new_particles.append((x, y, vy + 0.1, color, life))
    return new_particles

def classify_gesture(hand_landmarks, h, w):
    # Simple heuristic: thumbs up vs peace sign
    # Landmarks index as per MediaPipe:
    # 4: thumb_tip, 8: index_tip, 12: middle_tip, 16: ring_tip, 20: pinky_tip
    coords = lambda i: np.array([hand_landmarks.landmark[i].x * w, hand_landmarks.landmark[i].y * h])
    thumb_tip = coords(4)
    thumb_ip = coords(3)
    index_tip = coords(8)
    middle_tip = coords(12)
    ring_tip = coords(16)
    pinky_tip = coords(20)

    # Vector differences
    def is_extended(tip, pip):
        return tip[1] < pip[1]  # y decreases upward

    thumb_up = thumb_tip[1] < thumb_ip[1]  # thumb is up
    other_fingers_folded = not is_extended(index_tip, coords(6)) and not is_extended(middle_tip, coords(10)) and not is_extended(ring_tip, coords(14)) and not is_extended(pinky_tip, coords(18))
    if thumb_up and other_fingers_folded:
        return "thumbs_up"

    # Peace sign: index & middle extended, ring and pinky folded
    if is_extended(index_tip, coords(6)) and is_extended(middle_tip, coords(10)) and (not is_extended(ring_tip, coords(14))) and (not is_extended(pinky_tip, coords(18))):
        return "peace"

    return None

def create_badge(snapshot, name=CLUB_NAME, qr_img=None):
    h, w = snapshot.shape[:2]
    badge = np.zeros((h + 120, w + 20, 3), dtype=np.uint8) + 30  # dark background
    badge[10:10+h, 10:10+w] = snapshot
    cv2.putText(badge, f"Welcome to {name}!", (15, h + 35), cv2.FONT_HERSHEY_BOLD, 1, (255, 255, 255), 2)
    if qr_img is not None:
        qh, qw = qr_img.shape[:2]
        badge[h + 5:h + 5 + qh, w + 10 - qw//2:w + 10 - qw//2 + qw] = qr_img
        cv2.putText(badge, "Scan to Join", (15, h + 65 + qh - 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (200, 200, 200), 1)
    return badge

# ---------- Main Loop ----------
def main():
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Cannot open webcam.")
        return

    model = YOLO("yolov8n.pt")  # lightweight pretrained model

    mp_hands = mp.solutions.hands
    hands = mp_hands.Hands(static_image_mode=False,
                           max_num_hands=2,
                           min_detection_confidence=0.5,
                           min_tracking_confidence=0.5)
    mp_draw = mp.solutions.drawing_utils

    last_trigger_time = 0
    badge_show_until = 0
    current_badge = None
    confetti_particles = []

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.flip(frame, 1)  # mirror
        h, w = frame.shape[:2]

        # YOLO detection (person or any object)
        results = model(frame, verbose=False)[0]
        # Draw bounding boxes
        for box in results.boxes:
            cls_id = int(box.cls[0])
            conf = float(box.conf[0])
            label = model.names.get(cls_id, str(cls_id))
            if conf < 0.3:
                continue
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            cv2.rectangle(frame, (x1, y1), (x2, y2), (80, 180, 255), 2)
            cv2.putText(frame, f"{label} {conf:.2f}", (x1, y1 - 6), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1)

        # MediaPipe hand detection
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        hand_res = hands.process(rgb)
        gesture_detected = None
        if hand_res.multi_hand_landmarks:
            for handLms in hand_res.multi_hand_landmarks:
                mp_draw.draw_landmarks(frame, handLms, mp_hands.HAND_CONNECTIONS)
                gesture = classify_gesture(handLms, h, w)
                if gesture:
                    gesture_detected = gesture
                    cv2.putText(frame, f"Gesture: {gesture}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 255, 0), 2)
                    break  # prioritize first detected meaningful gesture

        # Trigger on gesture if person is present (simplified: if any 'person' box exists)
        person_present = any(int(box.cls[0]) == 0 for box in results.boxes)  # YOLO class 0 is 'person' in COCO
        now = time.time()
        if person_present and gesture_detected in ("thumbs_up", "peace") and now - last_trigger_time > 2:
            last_trigger_time = now
            # Create confetti burst
            confetti_particles = []
            for _ in range(CONFETTI_COUNT):
                x = random.uniform(0, w)
                y = random.uniform(-50, 0)
                vy = random.uniform(2, 5)
                color = (random.randint(100,255), random.randint(100,255), random.randint(100,255))
                life = random.randint(20, 40)
                confetti_particles.append((x, y, vy, color, life))
            # Generate badge snapshot
            snap = frame.copy()
            qr_img = generate_qr_image(SIGNUP_URL)
            badge = create_badge(snap, CLUB_NAME, qr_img)
            current_badge = badge
            badge_show_until = now + BADGE_DISPLAY_SECONDS

        # Draw confetti if active
        if confetti_particles:
            confetti_particles = draw_confetti(frame, confetti_particles)

        # Overlay welcome text if person present
        if person_present:
            cv2.putText(frame, f"Hey there! Give a thumbs-up to join {CLUB_NAME}", (10, h - 20),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 215, 0), 2)

        # Display badge if recent
        if current_badge is not None and now < badge_show_until:
            bh, bw = current_badge.shape[:2]
            # top-right corner
            x_offset = w - bw - 10
            y_offset = 10
            frame[y_offset:y_offset+bh, x_offset:x_offset+bw] = current_badge
        elif now >= badge_show_until:
            current_badge = None  # clear

        # Optional: small flashing prompt when peace sign
        if gesture_detected == "peace":
            cv2.putText(frame, "Peace! 🎉 Special offer: Free Sticker", (10, 60),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (100, 255, 100), 2)

        cv2.imshow("Club Fair Attraction", frame)
        key = cv2.waitKey(1) & 0xFF
        if key == 27:  # ESC
            break
        if key == ord("s") and current_badge is not None:
            # save badge
            fname = f"badge_{int(time.time())}.png"
            cv2.imwrite(fname, current_badge)
            print(f"Saved badge to {fname}")

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()

100%|██████████| 6.25M/6.25M [00:00<00:00, 27.2MB/s]


AttributeError: module 'cv2' has no attribute 'FONT_HERSHEY_BOLD'

: 